In [1]:
import json
with open('term_frequency.json') as f:
    top_terms = json.load(f)

In [2]:
len(top_terms)

2731

In [3]:
top_terms = {k: {} for k in top_terms}

In [4]:
from models import *
for post in Post.select():
    if not post.message:
        continue
    for sentence in post.message.split('\t'):
        for word in sentence.split(' '):
            if word in top_terms:
                k = post.time[0:10]
                if k in top_terms[word]:
                    top_terms[word][k] += 1
                else:
                    top_terms[word][k] = 1

In [5]:
from collections import Counter
times = []
for post in Post.select():
    if post.message:
        times.append(post.time[0:10])
time_counts = Counter(times)
for word in top_terms:
    for time in top_terms[word]:
        top_terms[word][time] /= time_counts[time]

In [6]:
from datetime import datetime
import pandas as pd
start = datetime(2015, 10, 17)
end = datetime(2016, 1, 16)
index = pd.date_range(start, end)

In [7]:
top_terms_with_peak = {}
one_day = pd.Timedelta('1 days')
for k in top_terms:
    s = pd.Series(top_terms[k])
    s.index = pd.DatetimeIndex(s.index)
    s = s.reindex(index, fill_value=0)
    peak = s[s > (s.mean() + 2 * s.std())]
    for d1 in peak.index:
        d2 = d1 + one_day
        d3 = d2 + one_day
        if d2 in peak.index and d3 in peak.index:
            top_terms_with_peak[k] = s
            print(k, end=' ')
            break

朱立倫 總統 選票 監督 反對 選 不斷 投票 夜 票 國際 穩定 1月 16日 消息 傷害 賠償 南 1 14 懇請 公開 現狀 下 中國 28 施政 一定 天 不要 競選 公車 困難 標準 能源 辯論 自信 開心 冷 元旦 堅定 關懷 部落 辯論會 抗議 進行 非 技術 31 生產 22 倒數 憲政 報告 即將 人選 交流 深刻 登記 頂 人物 經營 何 新加坡 恐怖 證所稅 年金 預計 團體 簽署 多少 取消 領導人 食品 衝 尊嚴 政黨票 拜託 黃安 選情 馬習會 表 馬 黑箱 食安 二十 柯 關 陳建仁 抹黑 11月 等於 廠 投給 過半 工人 出力 馬習 屬 聲明 馬英九 會面 習近平 協議 見面 停止 安排 握 鬥陣 叫做 舉行 聖誕 故宮 南院 前往 KMT 資產 陣營 ② 交換 頂新 藝人 知 購 破壞 地位 ① 黑心 軍宅 無罪 板橋區 116 

In [8]:
len(top_terms_with_peak)

132

In [17]:
meaningful = {'KMT', '①', '②', '中國', '交換', '交流', '人物', '人選', '倒數', '停止', '傷害', '冷', '出力',
 '前往', '協議', '南院', '即將', '反對', '取消', '叫做', '困難', '國際', '團體', '地位', '堅定', '報告', '夜', '安排',
 '尊嚴', '屬', '工人', '年金', '廠', '恐怖', '憲政', '懇請', '技術', '投票', '投給', '抗議', '抹黑', '拜託', '握',
 '政黨票', '故宮', '新加坡', '會面', '朱立倫', '標準', '消息', '無罪', '現狀', '登記', '監督', '破壞', '票', '穩定',
 '簽署', '經營', '總統', '習近平', '聲明', '能源', '自信', '舉行', '藝人', '衝', '表', '見面', '證所稅', '賠償', '購',
 '軍宅', '辯論', '辯論會', '進行', '過半', '選票', '部落', '關', '陳建仁', '頂', '頂新', '領導人', '食品', '食安', '馬',
 '馬習', '馬習會', '馬英九', '鬥陣', '黃安', '黑心', '黑箱'}
len(meaningful)

94

In [ ]:
from matplotlib import pyplot
group = {}
for k, v in top_terms_with_peak.items():
    if k in meaningful:
        v.plot()
        pyplot.show()
        option = ''
        while not option.isdecimal():
            option = input(k + ' ')
        group[k] = int(option)
        print(group)